## 1. Data Preparation & Filtering

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

DATA_DIR = Path("Trackman CSVs")   

In [2]:
import pandas as pd
import numpy as np

# -------------------------------
# Load the two datasets
# -------------------------------
ncaa_df = pd.read_csv("all_games.csv")
ucsd_df = pd.read_csv("ucsd_fall_pitching_data.csv")  # <-- match your exact filename on the left

# -------------------------------
# Backward-compatible aliases
# (so the rest of the notebook works)
# -------------------------------
all_df = ncaa_df.copy()      # NCAA-wide (used for baselines / percentiles / betas)
ucsd_all = ucsd_df.copy()    # UCSD-only (used for your tables / rankings)

print("Loaded NCAA rows:", len(all_df), "| UCSD rows:", len(ucsd_all))
display(all_df.head(3))
display(ucsd_all.head(3))

/var/folders/zh/lt350y8j697d9gr_ryq4y0gm0000gn/T/ipykernel_31082/756047426.py:7: DtypeWarning: Columns (91,92,93) have mixed types. Specify dtype option on import or set low_memory=False.
  ncaa_df = pd.read_csv("all_games.csv")


Loaded NCAA rows: 1501313 | UCSD rows: 7268


,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,...,ThrowTrajectoryZc1,ThrowTrajectoryZc2,PitchReleaseConfidence,PitchLocationConfidence,PitchMovementConfidence,HitLaunchConfidence,HitLandingConfidence,CatcherThrowCatchConfidence,CatcherThrowReleaseConfidence,CatcherThrowLocationConfidence
0,1,2025-02-14,18:04:47.71,1,1,"Stuprich, Brennan",1.000099e+09,Right,SOU_LIO,"Isom-McCall, Charlie",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
1,2,2025-02-14,18:05:00.16,1,2,"Stuprich, Brennan",1.000099e+09,Right,SOU_LIO,"Isom-McCall, Charlie",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
2,3,2025-02-14,18:05:14.48,1,3,"Stuprich, Brennan",1.000099e+09,Right,SOU_LIO,"Isom-McCall, Charlie",...,-78.35097,12.46061,High,High,High,NaN,NaN,Medium,Medium,Low


,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,...,ThrowTrajectoryZc1,ThrowTrajectoryZc2,PitchReleaseConfidence,PitchLocationConfidence,PitchMovementConfidence,HitLaunchConfidence,HitLandingConfidence,CatcherThrowCatchConfidence,CatcherThrowReleaseConfidence,CatcherThrowLocationConfidence
0,1,2025-11-01,11:35:12.81,1.0,1.0,"Gregson, Niccolas",1000039282,Right,CSD_TRI,"Crossland, Michael",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
1,2,2025-11-01,11:35:34.61,1.0,2.0,"Gregson, Niccolas",1000039282,Right,CSD_TRI,"Crossland, Michael",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
2,3,2025-11-01,11:36:01.08,1.0,3.0,"Gregson, Niccolas",1000039282,Right,CSD_TRI,"Crossland, Michael",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN


In [3]:
ncaa_df.groupby(['Pitcher', 'PitcherTeam']).size().sort_values(ascending=False)

Pitcher            PitcherTeam
Grinsell, Grayson  ORE_DUC        1486
Lockwood, Connor   VAL_CRU        1343
Hugus, Griffin     MIA_HUR        1331
Wright, Davis      WCC            1308
Jacobs, Ben        ARI_SUN        1284
                                  ... 
Shaw, Bradyn       AKR_ZIP           1
Smith, Caleb       HAR_SIM           1
Bevis, Blake       BAL_CAR           1
Monheimer, Riley   LIN_WIL1          1
Hervat, Jake       SLC_CCB           1
Length: 7332, dtype: int64

## 2. Pitch-Type–Specific Release Variability Metrics

In [4]:
df = all_df.copy()

def pick_col(candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

PITCHER_COL = pick_col(["Pitcher", "PitcherName", "PitcherNameFull"])
TEAM_COL    = pick_col(["PitcherTeam", "Team", "Pitcher Team", "PitcherTeamAbbrev"])
PITCHTYPE_COL = pick_col(["TaggedPitchType", "PitchType", "AutoPitchType"])

HRA_COL = pick_col(["HorzRelAngle"])
VRA_COL = pick_col(["VertRelAngle"])

# Horizontal + vertical release POINTS (usually X and Z)
RELX_COL = pick_col(["RelSide"])
RELZ_COL = pick_col(["RelHeight"])

print("Pitcher:", PITCHER_COL)
print("Team:", TEAM_COL)
print("PitchType:", PITCHTYPE_COL)
print("HRA:", HRA_COL)
print("VRA:", VRA_COL)
print("RelX:", RELX_COL)
print("RelZ:", RELZ_COL)

req = [PITCHER_COL, TEAM_COL, PITCHTYPE_COL, HRA_COL, VRA_COL, RELX_COL, RELZ_COL]
missing = [r for r in req if r is None]
if missing:
    raise ValueError("Missing required columns. Fix candidates list above. Missing: " + str(missing))

# Keep only what we need + clean
df = df.dropna(subset=[PITCHER_COL, TEAM_COL, PITCHTYPE_COL, HRA_COL, VRA_COL, RELX_COL, RELZ_COL]).copy()
df[PITCHTYPE_COL] = df[PITCHTYPE_COL].astype(str).str.strip()
df[PITCHER_COL]   = df[PITCHER_COL].astype(str).str.strip()
df[TEAM_COL]      = df[TEAM_COL].astype(str).str.strip()

df.head()

Pitcher: Pitcher
Team: PitcherTeam
PitchType: TaggedPitchType
HRA: HorzRelAngle
VRA: VertRelAngle
RelX: RelSide
RelZ: RelHeight


,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,...,ThrowTrajectoryZc1,ThrowTrajectoryZc2,PitchReleaseConfidence,PitchLocationConfidence,PitchMovementConfidence,HitLaunchConfidence,HitLandingConfidence,CatcherThrowCatchConfidence,CatcherThrowReleaseConfidence,CatcherThrowLocationConfidence
0,1,2025-02-14,18:04:47.71,1,1,"Stuprich, Brennan",1.000099e+09,Right,SOU_LIO,"Isom-McCall, Charlie",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
1,2,2025-02-14,18:05:00.16,1,2,"Stuprich, Brennan",1.000099e+09,Right,SOU_LIO,"Isom-McCall, Charlie",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
2,3,2025-02-14,18:05:14.48,1,3,"Stuprich, Brennan",1.000099e+09,Right,SOU_LIO,"Isom-McCall, Charlie",...,-78.35097,12.46061,High,High,High,NaN,NaN,Medium,Medium,Low
3,4,2025-02-14,18:05:39.46,2,1,"Stuprich, Brennan",1.000099e+09,Right,SOU_LIO,"Edwards, Kameron",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
4,5,2025-02-14,18:05:53.55,2,2,"Stuprich, Brennan",1.000099e+09,Right,SOU_LIO,"Edwards, Kameron",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN


## NCAA-Wide Spread Table

In [5]:
MIN_PITCHES_PER_TYPE = 0  # adjust if you want (prevents noisy tiny samples)

g = df.groupby([PITCHER_COL, PITCHTYPE_COL])

spread = g.agg(
    n_pitches=(PITCHTYPE_COL, "size"),
    sd_hra=(HRA_COL, "std"),
    sd_vra=(VRA_COL, "std"),
    sd_relx=(RELX_COL, "std"),
    sd_relz=(RELZ_COL, "std"),
).reset_index()

spread = spread[spread["n_pitches"] >= MIN_PITCHES_PER_TYPE].dropna()
spread.head()

,Pitcher,TaggedPitchType,n_pitches,sd_hra,sd_vra,sd_relx,sd_relz
0,"Abad, Adrian",Sinker,31,0.829183,1.192912,0.196204,0.118152
1,"Abad, Adrian",Slider,17,1.254848,1.380489,0.219038,0.119576
2,"Abbadessa, Jude",ChangeUp,7,1.005730,1.908953,0.290681,0.136591
3,"Abbadessa, Jude",Fastball,174,0.837360,0.954544,0.216094,0.114650
4,"Abbadessa, Jude",Sinker,20,0.751234,1.019078,0.300658,0.126558


## NCAA Percentile-Based Command Scores

In [6]:
# Percentile-based command (higher = better)
for col in ["sd_hra", "sd_vra", "sd_relx", "sd_relz"]:
    spread[f"{col}_pct"] = 1 - spread.groupby(PITCHTYPE_COL)[col].rank(pct=True)

## Regression Model for Beta Weights

In [7]:
# Select Features + Target (Plate Location)

# Features (predictors)
FEATURES = [
    "VertRelAngle",   # VRA
    "HorzRelAngle",   # HRA
    "RelHeight",      # vRel
    "RelSide"         # hRel
]

# Targets (plate location)
TARGETS = [
    "PlateLocHeight",  # Z location
    "PlateLocSide"     # X location
]

model_df = df.dropna(subset=FEATURES + TARGETS)

X = model_df[FEATURES]
y = model_df[TARGETS]

In [8]:
# Test/Train Split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [9]:
# Linear Regression (Kirby-style)

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

y_pred = lin_reg.predict(X_test)

print("MSE:", mean_squared_error(y_test, y_pred))
print("R²:", r2_score(y_test, y_pred))

MSE: 0.6286337962161277
R²: 0.3795154898604312


In [10]:
# Extract Beta Weights

beta_df = pd.DataFrame(
    lin_reg.coef_,
    columns=FEATURES,
    index=["PlateSide", "PlateHeight"]
)

beta_df

,VertRelAngle,HorzRelAngle,RelHeight,RelSide
PlateSide,0.389138,-0.029815,0.679094,-0.035437
PlateHeight,-0.057208,0.524754,-0.074106,0.697170


In [11]:
beta_weights = beta_df.abs().mean(axis=0)
beta_weights = beta_weights / beta_weights.sum()  # normalize

beta_weights

VertRelAngle    0.179492
HorzRelAngle    0.223012
RelHeight       0.302889
RelSide         0.294608
dtype: float64

## Build Pitch Command Spread (Raw SDs)

In [12]:
import numpy as np

# --- 1) Identify the correct release point columns in YOUR CSV ---
# Trackman commonly uses RelHeight (vertical release) and RelSide (horizontal release)
CAND_VREL = ["RelHeight", "ReleaseHeight", "release_pos_z", "RelZ", "vRel"]
CAND_HREL = ["RelSide", "ReleaseSide", "release_pos_x", "RelX", "hRel"]

vrel_col = next((c for c in CAND_VREL if c in df.columns), None)
hrel_col = next((c for c in CAND_HREL if c in df.columns), None)

print("Using vRel column:", vrel_col)
print("Using hRel column:", hrel_col)

if vrel_col is None or hrel_col is None:
    raise ValueError(
        f"Could not find release point columns. "
        f"Columns in df include: {list(df.columns)[:40]} ..."
    )

# --- 2) Compute SDs per (Pitcher, PitchType) ---
PITCHER_COL = "Pitcher"
PITCHTYPE_COL = "TaggedPitchType"

spread = (
    df.groupby([PITCHER_COL, PITCHTYPE_COL])
      .agg(
          sd_vra=("VertRelAngle", lambda s: s.std(ddof=1)),
          sd_hra=("HorzRelAngle", lambda s: s.std(ddof=1)),
          sd_vrel=(vrel_col, lambda s: s.std(ddof=1)),
          sd_hrel=(hrel_col, lambda s: s.std(ddof=1)),
          n=("VertRelAngle", "size")
      )
      .reset_index()
)

# (Optional but recommended) drop tiny sample sizes to reduce noise
MIN_PITCHES_PER_TYPE = 0
spread = spread[spread["n"] >= MIN_PITCHES_PER_TYPE].copy()

# --- 3) Z-score each SD metric within pitch type (NCAA-wide benchmark) ---
def z_by_pitchtype(series):
    mu = series.mean()
    sd = series.std(ddof=0)
    return (series - mu) / sd if sd != 0 else np.nan

spread["z_sd_vra"]  = spread.groupby(PITCHTYPE_COL)["sd_vra"].transform(z_by_pitchtype)
spread["z_sd_hra"]  = spread.groupby(PITCHTYPE_COL)["sd_hra"].transform(z_by_pitchtype)
spread["z_sd_vrel"] = spread.groupby(PITCHTYPE_COL)["sd_vrel"].transform(z_by_pitchtype)
spread["z_sd_hrel"] = spread.groupby(PITCHTYPE_COL)["sd_hrel"].transform(z_by_pitchtype)

spread.head()

Using vRel column: RelHeight
Using hRel column: RelSide


,Pitcher,TaggedPitchType,sd_vra,sd_hra,sd_vrel,sd_hrel,n,z_sd_vra,z_sd_hra,z_sd_vrel,z_sd_hrel
0,"Abad, Adrian",Sinker,1.192912,0.829183,0.118152,0.196204,31,0.926395,-0.290006,0.032746,0.169239
1,"Abad, Adrian",Slider,1.380489,1.254848,0.119576,0.219038,17,0.687510,0.406221,-0.133067,0.152883
2,"Abbadessa, Jude",ChangeUp,1.908953,1.005730,0.136591,0.290681,7,1.893653,-0.050870,0.047106,0.594430
3,"Abbadessa, Jude",Fastball,0.954544,0.837360,0.114650,0.216094,174,-0.222635,-0.489315,-0.186771,0.079375
4,"Abbadessa, Jude",Sinker,1.019078,0.751234,0.126558,0.300658,20,0.388122,-0.528884,0.131199,0.930157


## NCAA-Normalized Release Consistency Percentiles

In [13]:
# ============================================================
# Convert release SDs to percentiles (lower SD = better)
# Done per pitch type (NCAA-normalized)
# Creates: sd_vra_pct, sd_hra_pct, sd_vrel_pct, sd_hrel_pct
# ============================================================

PCT_COLS = {
    "sd_vra":  "sd_vra_pct",
    "sd_hra":  "sd_hra_pct",
    "sd_vrel": "sd_vrel_pct",
    "sd_hrel": "sd_hrel_pct",
}

missing_raw = [c for c in PCT_COLS.keys() if c not in spread.columns]
if missing_raw:
    raise ValueError(f"Missing raw SD columns needed for percentiles: {missing_raw}")

for raw_col, pct_col in PCT_COLS.items():
    # rank within each pitch type; invert so lower SD = higher percentile (better)
    spread[pct_col] = 1 - spread.groupby(PITCHTYPE_COL)[raw_col].rank(pct=True)

# round for display (hundredths)
for c in PCT_COLS.values():
    spread[c] = spread[c].round(2)

display(spread[list(PCT_COLS.values())].describe())

,sd_vra_pct,sd_hra_pct,sd_vrel_pct,sd_hrel_pct
count,29727.000000,29727.000000,29727.000000,29727.000000
mean,0.499765,0.499765,0.499765,0.499765
std,0.288716,0.288716,0.288716,0.288716
min,0.000000,0.000000,0.000000,0.000000
25%,0.250000,0.250000,0.250000,0.250000
50%,0.500000,0.500000,0.500000,0.500000
75%,0.750000,0.750000,0.750000,0.750000
max,1.000000,1.000000,1.000000,1.000000


## Random Forest -> Feature Importance Betas

In [14]:
# ================================
# Random Forest: Feature Importances (Beta Weights)
# (nonlinear; already normalized to sum to 1)
# ================================

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import pandas as pd

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

rf_regressor = RandomForestRegressor(
    n_estimators=300,
    random_state=42,
    n_jobs=-1,
    min_samples_leaf=5,
    max_features="sqrt"   # <-- FIX: don't use "auto"
)

rf_regressor.fit(X_train, y_train)

rf_importance = pd.Series(
    rf_regressor.feature_importances_,
    index=FEATURES
).sort_values(ascending=False)

# already sums to 1, but keep for safety
beta_weights = rf_importance / rf_importance.sum()

print("RF importances sum:", rf_importance.sum())
display(rf_importance)

print("Beta weights sum:", beta_weights.sum())
display(beta_weights)

RF importances sum: 0.9999999999999999


HorzRelAngle    0.328630
VertRelAngle    0.297787
RelSide         0.215161
RelHeight       0.158421
dtype: float64

Beta weights sum: 1.0000000000000002


HorzRelAngle    0.328630
VertRelAngle    0.297787
RelSide         0.215161
RelHeight       0.158421
dtype: float64

## TriKirby Equation - Composite Pitch Command Metric

In [15]:
# TriKirby Equation (percentile-based, higher = better)

betas = {
    "sd_vra_pct": beta_weights["VertRelAngle"],
    "sd_hra_pct": beta_weights["HorzRelAngle"],
    "sd_vrel_pct": beta_weights["RelHeight"],
    "sd_hrel_pct": beta_weights["RelSide"],
}

spread["TriKirby"] = (
    betas["sd_vra_pct"] * spread["sd_vra_pct"] +
    betas["sd_hra_pct"] * spread["sd_hra_pct"] +
    betas["sd_vrel_pct"] * spread["sd_vrel_pct"] +
    betas["sd_hrel_pct"] * spread["sd_hrel_pct"]
).round(3)

display(
    spread[
        ["Pitcher", "TaggedPitchType", "TriKirby",
         "sd_vra_pct", "sd_hra_pct", "sd_vrel_pct", "sd_hrel_pct"]
    ].head()
)

,Pitcher,TaggedPitchType,TriKirby,sd_vra_pct,sd_hra_pct,sd_vrel_pct,sd_hrel_pct
0,"Abad, Adrian",Sinker,0.364,0.11,0.67,0.36,0.25
1,"Abad, Adrian",Slider,0.266,0.17,0.26,0.48,0.25
2,"Abbadessa, Jude",ChangeUp,0.247,0.04,0.47,0.36,0.11
3,"Abbadessa, Jude",Fastball,0.552,0.58,0.73,0.50,0.28
4,"Abbadessa, Jude",Sinker,0.391,0.24,0.77,0.31,0.08


In [20]:
# ================================
# NCAA D1: Top/Bottom 20 TriKirby per Pitch Type (min pitches = 30)
# Requires: spread dataframe with TriKirby already computed
# ================================

MIN_PITCHES = 30
TOP_N = 10

PITCHER_COL   = "Pitcher"
PITCHTYPE_COL = "TaggedPitchType"
TRIKIRBY_COL  = "TriKirby"

# detect pitch count column (if you have one)
COUNT_COL = None
for c in ["n", "n_pitches", "count", "N"]:
    if c in spread.columns:
        COUNT_COL = c
        break

# optional team column (only included if present)
TEAM_COL = "PitcherTeam" if "PitcherTeam" in spread.columns else None

# build a clean base table
cols = [PITCHER_COL, PITCHTYPE_COL, TRIKIRBY_COL]
if COUNT_COL is not None:
    cols.insert(2, COUNT_COL)
if TEAM_COL is not None:
    cols.insert(1, TEAM_COL)

base = (
    spread.loc[:, cols]
    .dropna(subset=[TRIKIRBY_COL])
)

if COUNT_COL is not None:
    base = base[base[COUNT_COL] >= MIN_PITCHES]

# loop each pitch type and display top/bottom 20
pitch_types = sorted(base[PITCHTYPE_COL].dropna().unique())

print(f"Using MIN_PITCHES = {MIN_PITCHES} (count col = {COUNT_COL})")
print("Pitch types found:", pitch_types)

for pt in pitch_types:
    df_pt = base[base[PITCHTYPE_COL] == pt].copy()

    # sort high->low for top
    top20 = df_pt.sort_values(TRIKIRBY_COL, ascending=False).head(TOP_N)
    # sort low->high for bottom
    bot20 = df_pt.sort_values(TRIKIRBY_COL, ascending=True).head(TOP_N)

    print(f"\n=== {pt} — TOP {TOP_N} TriKirby (min {MIN_PITCHES} pitches) ===")
    display(top20.reset_index(drop=True))

    print(f"\n=== {pt} — BOTTOM {TOP_N} TriKirby (min {MIN_PITCHES} pitches) ===")
    display(bot20.reset_index(drop=True))

Using MIN_PITCHES = 30 (count col = n)
Pitch types found: ['ChangeUp', 'Curveball', 'Cutter', 'Fastball', 'FourSeamFastBall', 'Knuckleball', 'Sinker', 'Slider', 'Splitter', 'Sweeper', 'TwoSeamFastBall', 'Undefined']

=== ChangeUp — TOP 10 TriKirby (min 30 pitches) ===


,Pitcher,TaggedPitchType,n,TriKirby
0,"Clack, Miles",ChangeUp,43,0.870
1,"Beuter, Gavin",ChangeUp,30,0.864
2,"McCullough, Jon",ChangeUp,33,0.862
3,"Husak, Joe",ChangeUp,38,0.838
4,"Chrest, Evan",ChangeUp,39,0.819
5,"Barroqueiro, Jayden",ChangeUp,47,0.807
6,"Boccabello, Tanner",ChangeUp,32,0.799
7,"Rooney, Noah",ChangeUp,35,0.797
8,"Herrera, Tyler",ChangeUp,35,0.779
9,"Galy, Gavin",ChangeUp,67,0.776



=== ChangeUp — BOTTOM 10 TriKirby (min 30 pitches) ===


,Pitcher,TaggedPitchType,n,TriKirby
0,"Smith, Tyler",ChangeUp,34,0.042
1,"Cychosz, Isaac",ChangeUp,30,0.054
2,"Madariaga, Reid",ChangeUp,31,0.067
3,"Nelson, Drew",ChangeUp,166,0.071
4,"Miner, Jace",ChangeUp,41,0.079
5,"Bender, Pryce",ChangeUp,45,0.082
6,"Sexton, Landon",ChangeUp,58,0.103
7,"Lewis, Kyle",ChangeUp,35,0.106
8,"Brown, Dylan",ChangeUp,206,0.106
9,"Jibben, Payten",ChangeUp,37,0.113



=== Curveball — TOP 10 TriKirby (min 30 pitches) ===


...

,Pitcher,TaggedPitchType,n,TriKirby
0,"Maloney, Collin",TwoSeamFastBall,270,0.105
1,"Monaco, Dominic",TwoSeamFastBall,39,0.141
2,"Voth, Brant",TwoSeamFastBall,43,0.142
3,"Iglesias, Joey",TwoSeamFastBall,70,0.173
4,"West, Manning",TwoSeamFastBall,31,0.174
5,"Peteson, Sam",TwoSeamFastBall,38,0.175
6,"Arroyo, Chris",TwoSeamFastBall,49,0.181
7,"Van Der Lelie , Jelle",TwoSeamFastBall,131,0.188
8,"Turner , Kole",TwoSeamFastBall,220,0.197
9,"White, Caleb",TwoSeamFastBall,40,0.200



=== Undefined — TOP 10 TriKirby (min 30 pitches) ===


,Pitcher,TaggedPitchType,n,TriKirby
0,"Tollett, JR",Undefined,50,0.867
1,"Mabee, Oliver",Undefined,40,0.853
2,"Barquin, Blake",Undefined,43,0.827
3,"Merkel, Alex",Undefined,33,0.824
4,"Mankoski, Nathan",Undefined,57,0.818
5,"Whitmer, Marshall",Undefined,30,0.805
6,"Mertens, Nathan",Undefined,80,0.802
7,"Tejada, Anthony",Undefined,32,0.790
8,"Johnson, Nolan",Undefined,94,0.787
9,"Herrmann, Andrew",Undefined,73,0.786



=== Undefined — BOTTOM 10 TriKirby (min 30 pitches) ===


,Pitcher,TaggedPitchType,n,TriKirby
0,"Mitchell, Joey",Undefined,251,0.044
1,"Hunsaker, Riely",Undefined,181,0.054
2,"Kimball, Blake",Undefined,182,0.069
3,"Renuart, Jake",Undefined,66,0.076
4,"Neal, Austin",Undefined,85,0.079
5,"Arrichiello, Anthony",Undefined,80,0.080
6,"Cooksey, Braxton",Undefined,35,0.082
7,"Westfall, Derek",Undefined,44,0.100
8,"Dottavio, Dom",Undefined,65,0.109
9,"Posey, Jack",Undefined,82,0.117


## NCAA D1 Average TriKirby Index Score Per Pitch

In [33]:
# ============================================
# NCAA D1 Average TriKirby Score by Pitch Type
# ============================================

ncaa_pitch_averages = (
    spread
    .dropna(subset=["TriKirby"])
    .groupby("TaggedPitchType")
    .agg(
        NCAA_Avg_TriKirby=("TriKirby", "mean"),
        Num_Pitchers=("Pitcher", "nunique"),
        Num_Pitcher_Pitch_Pairs=("TriKirby", "count")
    )
    .sort_values("NCAA_Avg_TriKirby", ascending=False)
    .round(3)
    .reset_index()
)

display(ncaa_pitch_averages)


,TaggedPitchType,NCAA_Avg_TriKirby,Num_Pitchers,Num_Pitcher_Pitch_Pairs
0,Fastball,0.500,6758,6758
1,Slider,0.500,6226,6226
2,ChangeUp,0.500,5125,5125
3,Curveball,0.500,3730,3730
4,Cutter,0.500,1988,1988
5,FourSeamFastBall,0.500,1773,1773
6,Sinker,0.500,1845,1845
7,Undefined,0.500,1010,1010
8,TwoSeamFastBall,0.499,477,477
9,Splitter,0.499,439,439
